In [ ]:
#1.Enter Repo Name and ID
HUGGING_FACE_USER_NAME = "Tatvajsh"
model_name = "AHS_OPS_alpha_v_2.0"

In [ ]:
!pip install transformers[sentencepiece]


In [ ]:
#2.Install Necessary Libraries
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/peft.git git+https://github.com/huggingface/transformers.git

In [ ]:
#3.Load the Model with Adapter in GPU for INFERENCE
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = f"{HUGGING_FACE_USER_NAME}/{model_name}"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained("openlm-research/open_llama_3b_v2", device_map="auto",load_in_4bit=True)



In [ ]:
tokenizer = AutoTokenizer.from_pretrained(peft_model_id,use_fast=False)
# Load the Lora model
model1 = PeftModel.from_pretrained(model, peft_model_id,device_map="auto")
model1.resize_token_embeddings(len(tokenizer))
model1

In [ ]:

prompt1=f"""
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

    ### Instruction:

    the URL for the official website of the AHS Health Link Policies department is https://insite.albertahealthservices.ca/811/Page8024.aspx

    ### Response:

    """
tokenizer.tokenize(prompt1, return_tensors='pt',add_special_tokens=True)
# tokenizer

# BEST HYPERPARAMETERS FOR NOW:    

> outputs = model.generate(input_ids=batch["input_ids"].to("cuda"),
                            temperature=0.001,
                             max_new_tokens=150,
                             repetition_penalty=1.2,
                             output_scores=True,
                             top_k=90,
                             do_sample=True)


In [ ]:
#4.Generate the Text based upon user Question
from IPython.display import display, Markdown

def make_inference(instructions):

  prompt = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.\
    ### Instruction:{instructions}  \
    ### Response:  """
  prompt1=f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
    ### Instruction:
    {instructions}
    ### Response:
    """
  prompt2=f"Q:{instructions} \n A:"
  batch = tokenizer(prompt1, return_tensors='pt')
#   model.config.use_cache = True
  with torch.no_grad():
    outputs = model1.generate(input_ids=batch["input_ids"].to("cuda"),use_cache=False,output_attentions=False,min_new_tokens=60,temperature=0.2,max_new_tokens=512,repetition_penalty=1.2,output_scores=True,return_dict_in_generate=True,top_p=0.3,top_k=50,do_sample=True)
      # outputs = model1.generate(input_ids=batch["input_ids"].to("cuda"),use_cache=True,max_new_tokens=512,repetition_penalty=1.2,output_scores=True,return_dict_in_generate=True)

    # print(tokenizer.batch_decode(outputs.detach().cpu().numpy())[0])
  return tokenizer.batch_decode(outputs[0].detach().cpu().numpy(),skip_special_tokens=False)[0],outputs
  # display(Markdown(tokenizer.decode(output[0], skip_special_tokens=True)))

In [ ]:

import time
start_time = time.time()
instruction=" What does Operations oversee?"

prompt=f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
    ### Instruction:
    {instruction}
    ### Response:
    """
resp,outputs=make_inference(f"{instruction}")
end_time = time.time()
# Calculate the elapsed time
elapsed_time = end_time - start_time

print(f"Execution time: {elapsed_time} seconds")
print(resp)

In [ ]:




#6.OUTPUT FORMAT 1
answer=" ".join(resp.split("Response:")[1:])
print(answer)

In [ ]:


import numpy as np
transition_scores = model.compute_transition_scores(
    outputs.sequences, outputs.scores, normalize_logits=True
)
transition_scores
inputs=tokenizer(f"{prompt}", return_tensors='pt')
input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
generated_tokens = outputs.sequences[:, input_length:]
for tok, score in zip(generated_tokens[0], transition_scores[0]):
    # | token | token string | logits | probability
    print(f"| {tok:5d} | {tokenizer.decode(tok):8s} | {score.cpu().data.numpy():.3f} | {np.exp(score.cpu().data.numpy()):.2%}")

In [3]:
!pip -q install langchain huggingface_hub transformers sentence_transformers faiss-gpu accelerate
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/peft.git git+https://github.com/huggingface/transformers.git

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader,DirectoryLoader,TextLoader,CSVLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

In [5]:
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
model = AutoModelForCausalLM.from_pretrained(
        "openlm-research/open_llama_3b_v2",
        device_map="auto",
        torch_dtype=torch.float16,
        load_in_4bit=True
#         low_cpu_mem_usage=True
#         load_in_8bit=True,#original for best performance:        load_in_4bit=True

    )
tokenizer = AutoTokenizer.from_pretrained("openlm-research/open_llama_3b_v2",trust_remote_code=True,use_fast=False,padding=256,truncation=256)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [6]:
#1.Enter Repo Name and ID
HUGGING_FACE_USER_NAME = "Tatvajsh"
model_name1 = "AHS_OPS_alpha_v_2.0"
model_name2="AHS_WPCS_alpha_v_1.0"
peft_model_id = f"{HUGGING_FACE_USER_NAME}/{model_name1}"

In [7]:
from peft import PeftModel, PeftConfig
model1 = PeftModel.from_pretrained(model, peft_model_id,device_map="auto",torch_dtype=torch.float16, low_cpu_mem_usage=True)
# merged_model = model1.merge_and_unload()
# model1.load_adapter(peft_model_id, adapter_name="wpcs")
# model2 = PeftModel.from_pretrained(model, peft_model_id,device_map="auto",adapter_name="ops")


In [ ]:
model1.resize_token_embeddings(len(tokenizer))

In [ ]:
model1

In [ ]:
from peft import PeftModel, PeftConfig
model2 = PeftModel.from_pretrained(model, peft_model_id,device_map="auto",adapter_name="ops")


In [ ]:
model2


In [ ]:
model1.resize_token_embeddings(len(tokenizer))

In [ ]:
model1

In [4]:
pipe = pipeline("text-generation",
                model=model1,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 256,
                do_sample=True,
                top_k=30,
                num_return_sequences=2,
                temperature=0.50,
                # eos_token_id=tokenizer.eos_token_id,
                # use_cache=True,
                repetition_penalty=1.1,
                # output_scores=True,
                # return_dict_in_generate=True,
                # num_beams=15,
                )

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonForCausalLM', 'PLBartFo

In [ ]:
DATA_PATH="/kaggle/input/latest-dataset"
DB_FAISS_PATH="/kaggle/working/vectorstores/db_faiss"

#create vector database
def create_vector_db():
    #Create a DirectoryLoader to load all PDFs from the DATA_PATH. Use the PyPDFLoader to load each PDF.
    loader=DirectoryLoader(DATA_PATH,glob="*.txt",loader_cls=TextLoader)
    documents=loader.load()
    #shared overlapping text gives some continuity between chunks and context.
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=400,chunk_overlap=50)
    # text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=150)

    texts=text_splitter.split_documents(documents)# all the splitted text is here,text chunks
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                     model_kwargs={'device':'cuda'})#creating the embeddings
    db=FAISS.from_documents(texts,embeddings)#using this embedding model,create all the embedding and store it
    db.save_local(DB_FAISS_PATH)


create_vector_db()

In [7]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                       model_kwargs={'device': 'cuda'})

db = FAISS.load_local("/kaggle/working/vectorstores/db_faiss", embeddings)

In [8]:
from langchain import HuggingFacePipeline
from langchain import PromptTemplate,  LLMChain
# from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import CTransformers
from langchain.chains import RetrievalQA
llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0.30,'repetition_penalty':1.3,"num_beams":10})

In [9]:
custom_prompt_template="""Given the context information provided, please answer the user's question.

Context: {context}

Question: {question}
"""
custom_prompt_template1="""
Given the context information provided, please concisely and accurately answer the user's question. If the provided context is insufficient to determine an answer, please say so. Where appropriate, please provide any explanations or additional details that would be helpful to understanding the answer. Please do not speculate or fabricate information beyond what can be inferred from the given context.

### Context: {context}

### Question: {question}
### Answer: """
custom_prompt_template2="""
 Below is an instruction that describes a task. Write a response that appropriately completes the request from the given context
    ### Instruction:
    {question}
    ### Context:
    {context}
    ### Response:

    """
custom_prompt_template3="""
 Below is an instruction that describes a task. Write a response that appropriately completes the request from thr given context
    ### Instruction=>
    {question}
    ### Context=>
    {context}
    ### Response=>

    """
#concisely while including all relevant details. Reference any links, phone numbers, or email addresses that are relevant to the answer verbatim. If the question cannot be fully answered based on the given context, respond "I do not have enough context to fully answer this question. Please rephrase or provide more details about what you are looking for." Do not speculate or make up an answer.
# Concise, factual answer using relevant details and verbatim references from the provided context:
prompt = PromptTemplate(template=custom_prompt_template2,input_variables=[ 'question',"context"])
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                       chain_type='stuff',
                                       retriever=db.as_retriever(search_kwargs={'k': 3}),
                                       return_source_documents=True,
                                       chain_type_kwargs={'prompt': prompt}
                                       )

In [10]:
question = "Where can I find COVID-19 call handling documents?"
# question = "Where  the Manager on Call (MOC) Addiction and Mental Health Helpline can I find?"

from langchain import PromptTemplate,  LLMChain
res=qa_chain({'query': question})
# res=qa_chain.run(question)
print(res["result"])#Navigation Resources include Provincial Manager, Coordinators, a Web Master, and Clerical Support Staff. They maintain clinical and non-clinical content such as flowcharts, RightFax, EchoAccess, the Health Link intranet, and the Inform Alberta website. [Navigation Resources (https://insite.albertahealthservices.ca/811/Page15109.aspx)]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

COVID-19 call handling documents can be found [here](https://insite.albertahealthservices.ca/811/Page24484.aspx#doc-update).
The process for COVID vaccine escalation for people who are homebound can be found [here](https://share.albertahealthservices.ca/teams/HLA/Webdocs/Emerging%20Events/tms-811-covid-vaccine-escalation-homebound.pdf).
Work instruction documents related to COVID-19 can be found [here](https://insite.albertahealthservices.ca/811/Page24685.aspx#work-instruction-docs).
Testing protocols are available [here](https://insite.albertahealthservices.ca/811/Page24685.aspx#testing-protocols).
Work instruction documents related to COVID-19 testing can be found [here](https://insite.albertahealthservices.ca/811/Page24685.aspx#test-work-


In [11]:
res
#  The IT Acceptable Use Policy at AHS provides guidance and standards on the use of IT resources, including the Internet and electronic forms of communication (e-mail) for business purposes. It emphasizes the need for exercising careful judgment when using the Internet, intranet, e-mail, or other AHS IT resources.
# The URL for the Insulin Pump Therapy learning module is [here](https://myhealth.alberta.ca/learning/modules/insulin-pump-therapy).
# You can find the 'Collection, Access, Use, & Disclosure of Information 1112' policy [here](https://extranet.ahsnet.ca/teams/policydocuments/1/clp-ahs-pol-collection-access-use-disclosure-information.pdf).

{'query': 'Where can I find COVID-19 call handling documents?',
 'result': 'COVID-19 call handling documents can be found [here](https://insite.albertahealthservices.ca/811/Page24484.aspx#doc-update).\nThe process for COVID vaccine escalation for people who are homebound can be found [here](https://share.albertahealthservices.ca/teams/HLA/Webdocs/Emerging%20Events/tms-811-covid-vaccine-escalation-homebound.pdf).\nWork instruction documents related to COVID-19 can be found [here](https://insite.albertahealthservices.ca/811/Page24685.aspx#work-instruction-docs).\nTesting protocols are available [here](https://insite.albertahealthservices.ca/811/Page24685.aspx#testing-protocols).\nWork instruction documents related to COVID-19 testing can be found [here](https://insite.albertahealthservices.ca/811/Page24685.aspx#test-work-',
 'source_documents': [Document(page_content='COVID-19 call handling documents can be found [here](https://insite.albertahealthservices.ca/811/Page24484.aspx#doc-updat

In [11]:
merged_model = model1.merge_and_unload()

from huggingface_hub import notebook_login

notebook_login()

merged_model.push_to_hub("Tatvajsh/AHS_OPS_GGUF")